In [1]:
import keras.callbacks as cb
from keras.datasets import mnist
from keras.layers.core import Activation, Dense, Dropout
from keras.layers import LSTM
from keras.layers import Embedding
from keras.preprocessing import sequence


from keras.models import Sequential
from keras.optimizers import SGD,Adam
from keras.regularizers import l1, l2
from keras.utils import np_utils

%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import time
import pandas as pd


Using TensorFlow backend.


# Pre-processing

In [2]:
def PreprocessDataset():
    from sklearn import preprocessing
    data = pd.read_csv('Data/cleaned_data.csv',index_col=False,low_memory=False)
    #data = data.reindex(np.random.permutation(data.index))
    cols = data.columns
    x_columns = cols.drop("match_result")
    
    x = data[x_columns]
    y = data['match_result']
    
    train_max_row = int(data.shape[0]*0.9)
    
    x_train = x.iloc[:train_max_row]
    x_test = x.iloc[train_max_row:]
    
    y_train = y.iloc[:train_max_row]
    y_test = y.iloc[train_max_row:]
    
    y_train = np_utils.to_categorical(y_train)
    y_test = np_utils.to_categorical(y_test)
    
    ################Pre-processing###########
    min_max_scaler = preprocessing.MinMaxScaler()
    x_train = min_max_scaler.fit_transform(x_train)
    x_test = min_max_scaler.fit_transform(x_test)
    
    maxlen = 50

    print('Pad sequences (samples x time)')
    x_train = sequence.pad_sequences(x_train, maxlen=maxlen,dtype='float64')
    x_test = sequence.pad_sequences(x_test, maxlen=maxlen,dtype='float64')
    #x_train = x_train.reshape(19917,18,1)   
    #x_test = x_test.reshape(2213,18,1)

    print('X_train shape:', x_train.shape)
    print('X_test shape:', x_test.shape)
    
    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = PreprocessDataset()

Pad sequences (samples x time)
('X_train shape:', (19917, 50))
('X_test shape:', (2213, 50))


In [3]:
x_train[10]

array([ 0.02805611,  0.07861793,  0.10606061,  0.03951368,  0.09677419,
        0.16564417,  0.05439331,  0.07314629,  0.13043478,  0.02805611,
        0.07907908,  0.1736227 ,  0.03892944,  0.06450321,  0.12030075,
        0.02424364,  0.56363636,  0.55155556,  0.50237288,  0.47911111,
        0.37172414,  0.54884354,  0.66694444,  0.48727273,  0.53562654,
        0.46036036,  0.45373339,  0.74504505,  0.40890028,  0.57556536,
        0.52111486,  0.39987715,  1.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

In [4]:
'''
data = pd.read_csv('Data/cleaned_data.csv',index_col=False,low_memory=False)
data = data.reindex(np.random.permutation(data.index))
cols = data.columns
x_columns = cols.drop("match_result")
y = data['match_result']
x = data[x_columns]

type(x)
'''
y_train.shape


(19917, 3)

In [5]:
def DefineModel():

    activation_func = 'relu' 

    loss_function = 'categorical_crossentropy'
 
    dropout_rate = 0.4
    
    weight_regularizer = None

    learning_rate = 0.001
    
    nb_filter = 1
    filter_length = 3
    
    embedding_dims = 50
    max_features = 500
    maxlen = 50
    
    ## Initialize model.
    model = Sequential()
    
    model.add(Embedding(max_features, 128, dropout=0.2))
    print model.input_shape
    print model.output_shape


    ## First hidden layer with 'first_layer_width' neurons. 
    ## Also need to specify input dimension.
    ## 'Dense' means fully-connected.
    model.add(LSTM(128, dropout_W=0.4, dropout_U=0.2))  # try using a GRU instead, for fun

    
    print model.input_shape
    print model.output_shape
    

    #model.add(Flatten())

    # We add a vanilla hidden layer:
    
    model.add(Dense(64))
    model.add(Dropout(dropout_rate))
    model.add(Activation('relu'))
    
    
    ## Last layer has the same dimension as the number of classes
    model.add(Dense(3))
    ## For classification, the activation is softmax
    model.add(Activation('softmax'))
    ## Define optimizer. In this tutorial/codelab, we select SGD.
    ## You can also use other methods, e.g., opt = RMSprop()
    opt = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    #opt = SGD(lr=learning_rate, clipnorm=5.)
    ## Define loss function = 'categorical_crossentropy' or 'mean_squared_error'
    model.compile(loss=loss_function, optimizer=opt, metrics=["accuracy"])

    return model

In [6]:
def TrainModel(data=None, epochs=20):
    ################################################################
    # Activity 7 (Mini-batch):
    # Group A uses mini-batch of size 128.
    # batch = 128
    #
    # Group B uses mini-batch of size 256.
    # batch = 256
    # 
    # Group C uses mini-batch of size 512.
    # batch = 512
    ################################################################
    batch=32
    start_time = time.time()
    model = DefineModel()
    if data is None:
        print("Must provide data.")
        return
    x_train, x_test, y_train, y_test = data
    print('Start training.')
    ## Use the first 55,000 (out of 60,000) samples to train, last 5,500 samples to validate.
    history = model.fit(x_train[:17500], y_train[:17500], nb_epoch=epochs, batch_size=batch,
              validation_data=(x_train[17500:], y_train[17500:]))
    print("Training took {0} seconds.".format(time.time() - start_time))
    return model, history

In [7]:
trained_model, training_history = TrainModel(data=[x_train, x_test, y_train, y_test],epochs = 30)

(None, None)
(None, None, 128)
(None, None)
(None, 128)
Start training.


/home/dawei/anaconda2/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 17500 samples, validate on 2417 samples
Epoch 1/30
17500/17500 [==============================] - 56s - loss: 1.0721 - acc: 0.4549 - val_loss: 1.0709 - val_acc: 0.4398
Epoch 2/30
17500/17500 [==============================] - 57s - loss: 1.0684 - acc: 0.4556 - val_loss: 1.0690 - val_acc: 0.4398
Epoch 3/30
17500/17500 [==============================] - 53s - loss: 1.0678 - acc: 0.4556 - val_loss: 1.0698 - val_acc: 0.4398
Epoch 4/30
  768/17500 [>.............................] - ETA: 51s - loss: 1.0647 - acc: 0.4609

KeyboardInterrupt: 

In [8]:
def PlotHistory(train_value, test_value, value_is_loss_or_acc):
    f, ax = plt.subplots()
    ax.plot([None] + train_value, 'o-')
    ax.plot([None] + test_value, 'x-')
    ## Plot legend and use the best location automatically: loc = 0.
    ax.legend(['Train ' + value_is_loss_or_acc, 'Validation ' + value_is_loss_or_acc], loc = 0) 
    ax.set_title('Training/Validation ' + value_is_loss_or_acc + ' per Epoch')
    ax.set_xlabel('Epoch')
    ax.set_ylabel(value_is_loss_or_acc)  
    


In [9]:
    
PlotHistory(training_history.history['loss'], training_history.history['val_loss'], 'Loss')
PlotHistory(training_history.history['acc'], training_history.history['val_acc'], 'Accuracy')

NameError: name 'training_history' is not defined

In [ ]:
def drawWeightHistogram(x):
    ## the histogram of the data
    fig = plt.subplots()
    n, bins, patches = plt.hist(x, 50)
    plt.xlim(-0.5, 0.5)
    plt.xlabel('Weight')
    plt.ylabel('Count')
    zero_counts = (x == 0.0).sum()
    plt.title("Weight Histogram. Num of '0's: %d" % zero_counts)

In [ ]:
w1 = trained_model.layers[0].get_weights()[0].flatten()
drawWeightHistogram(w1)

In [ ]:
def TestModel(model=None, data=None):
    if model is None:
        print("Must provide a trained model.")
        return
    if data is None:
        print("Must provide data.")
        return
    x_test, y_test = data
    scores = model.evaluate(x_test, y_test)
    return scores

In [ ]:
test_score = TestModel(model=trained_model, data=[x_test, y_test])
print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))